#################### <br>
10/4일 필요한 작업 <br>
-> 1. 어떤 수화를 넣을 건지 리스트 작성, 수화 공부하기 <br>
-> 2. 수화 데이터 쌓을 때, 모션인식 제대로 하지 않으면 데이터 학습 시 오류 발생 -> 빈 배경에서 실행, 상반신 어느정도로 나와야 하나 테스트 <br>
-> 3. 데이터 쌓은 후 -> 모델 정확도 개선 <br>
######################



# 필요한 라이브러리 설치 및 import 
1. Import and Install Dependencies

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import font_manager,rc

font_path='c:\\data\\malgun.ttf'
font_name=font_manager.FontProperties(fname=font_path).get_name()
rc('font', family=font_name)

In [2]:
#GPU를 사용하는 tensorflow와 mediapipe 설치 

# mediapipe란 ? 주로 멀티 모달 오디오, 비디오 또는 모든 시계열 데이터를 구축하는 데 사용되는 프레임 워크입니다.

#using tensorflow with GPU, mediapipe detacting movements
#!pip install tensorflow==2.4.1 tensorflow-gpu==2.4.1 opencv-python mediapipe sklearn matplotlib

In [2]:
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
import time
import mediapipe as mp

# MP Holistic을 사용하여 pose, face, righthand, lefthand에 대한 <br>key point 생성하기 
2. Keypoints using MP Holistic


In [3]:
mp_holistic = mp.solutions.holistic 
#mp.solutions.holistic 이란 ? -> 실시간 모션인식 (얼굴,몸의 분절을 실시간으로 추적한다.)

#Live perception of simultaneous human pose, face landmarks, 
#and hand tracking in real-time on mobile devices can enable various modern life applications
#https://google.github.io/mediapipe/solutions/holistic.html



mp_drawing = mp.solutions.drawing_utils # Drawing utilities
# 점을 연결하여 선을 보여준다.

In [4]:
# 이미지를 추적하는 함수 만들기 
#이미지 -> RGB형식으로 만들어 모델에 대입한 결과를 results에 담고, 원상태로 만든 이미지와 results를 출력한다. 

def mediapipe_detection(image, model):   # 대상을 추적한다. 
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # COLOR CONVERSION BGR 2 RGB
    image.flags.writeable = False                  # Image is no longer writeable
    results = model.process(image)                 # Make prediction 모델에 대입 
    image.flags.writeable = True                   # Image is now writeable 
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) # COLOR COVERSION RGB 2 BGR
    return image, results

In [5]:
# 동영상에 점 찍기 

def draw_landmarks(image, results):  # 동영상의 frame과 동영상에서 받은 result를 받는다. 
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_CONTOURS) # 얼굴의 점을 연결하여 그린다.
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS) # POSE를 연결햐여 그린다.
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS) # 왼손 그리기
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS) # 오른손 그리기 

In [6]:
# 더 정확한 판단을 위해 각각의 노드에 두개의 점을 추가한다. 
#add aditional 2 spots to the image

def draw_styled_landmarks(image, results):
    # Draw face connections
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_CONTOURS, 
                             mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1),  
                             mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)
                             ) 
    # Draw pose connections
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                             ) 
    # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                             ) 
    # Draw right hand connections  
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                             ) 

In [7]:
########x#################
#안해도됨 
cap = cv2.VideoCapture(0)
# Set mediapipe model 
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():  # cap이 켜져있을 동안 

        # Read feed
        ret, frame = cap.read()  #화면을 읽어서 ret과 frame 에 넣는다 

        # Make detections
        image, results = mediapipe_detection(frame, holistic)  # frame에 있는 점들을 추적하여 result에 넣는다. 
        print(results)
        
        # Draw landmarks
        draw_styled_landmarks(image, results)  # 이미지와 result에 landmark를 찍는다

        # Show to screen
        cv2.imshow('OpenCV Feed', image) 

        # Break gracefully
        if cv2.waitKey(10) & 0xFF == ord('q'):  #q를 누르면 끝난다. 
            break
    cap.release()
    cv2.destroyAllWindows()

<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.soluti

<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.soluti

<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.soluti

# 3. Keypoint 값을 넘파이 형태로 추출하기 
3. Extract Keypoint Values

results.pose_landmarks.landmark -> [res.x, res.y, res.z, res.visibility]으로 구성 <br>
그외는 <br>
results.face_landmarks.landmark -> [res.x, res.y, res.z]으로 구성 


In [8]:
##위의 코드를 간단하게 하고 pose, face, lh,rh로 된 각각의  numpy 배열로 만들기 
# 넘파이배열 -> flatten 하기 
# 입력된 값이 없다면 0으로 채우기 (x,y,z 세개만큼, x3해준다.)

# if there's data in landmark, make them into numpy array. else fill with 0
pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(132)
face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(1404)
lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)

In [9]:
## 위에서 만든 코드를 함수로 만들기 

# pose, face, lh, rh의 순서로 저장한다. 
# np.concatenate 를 하여 넘파이 배열을 하나로 합친다. 

def extract_keypoints(results):
    pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4)
    face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(468*3)
    lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    return np.concatenate([pose, face, lh, rh])

#extract_keypoints(results).shape 
# (1662,) -> 468*3 + 33*4 + 21*3 + 21*3 

In [180]:
#no hand detection이 많이 없는 영상 뽑기
#영상 detection
video = 'C://Users//User//sign//video3000//KETI_SL_0000005382.avi'

cap = cv2.VideoCapture(video)

# Set mediapipe model 
a = 0
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():  # cap이 켜져있을 동안 
        a+= 1

        # Read feed
        ret, frame = cap.read()  #화면을 읽어서 ret과 frame 에 넣는다 
        if ret == True:


            # Make detections
            image, results = mediapipe_detection(frame, holistic)  # frame에 있는 점들을 추적하여 result에 넣는다. 
            print(results.right_hand_landmarks)
            if results.right_hand_landmarks == None:
                cv2.putText(image, 'no hand detection', (120,200), 
                cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255, 0), 4, cv2.LINE_AA)

            # Draw landmarks
            draw_styled_landmarks(image, results)  # 이미지와 result에 landmark를 찍는다

            # Show to screen
            cv2.imshow('OpenCV Feed', image)
            
            if cv2.waitKey(400) & 0xFF == ord('q'):  #q를 누르면 끝난다. 
                break



        # Break gracefully
        else:
            break
            
            
    
cap.release()
cv2.destroyAllWindows()

None
None
None
None
None
None
None
None
None
landmark {
  x: 0.37820568680763245
  y: 0.8216931819915771
  z: 0.0
}
landmark {
  x: 0.39807477593421936
  y: 0.7929674386978149
  z: 0.0007317987037822604
}
landmark {
  x: 0.42263638973236084
  y: 0.7840684652328491
  z: -0.0004218852845951915
}
landmark {
  x: 0.440056711435318
  y: 0.7820811867713928
  z: -0.003113700542598963
}
landmark {
  x: 0.4498223066329956
  y: 0.7757480144500732
  z: -0.004511813633143902
}
landmark {
  x: 0.44625064730644226
  y: 0.8585876822471619
  z: 0.00579563295468688
}
landmark {
  x: 0.44717881083488464
  y: 0.8880247473716736
  z: 0.005094322841614485
}
landmark {
  x: 0.43695858120918274
  y: 0.8850135803222656
  z: 0.006534401793032885
}
landmark {
  x: 0.4309878647327423
  y: 0.8746729493141174
  z: 0.007950523868203163
}
landmark {
  x: 0.43620577454566956
  y: 0.887652575969696
  z: 0.0024391005281358957
}
landmark {
  x: 0.4368475675582886
  y: 0.9224557280540466
  z: 0.003117014653980732
}
landm

landmark {
  x: 0.41776934266090393
  y: 0.35193800926208496
  z: 0.0
}
landmark {
  x: 0.41593968868255615
  y: 0.3193286657333374
  z: 0.009892704896628857
}
landmark {
  x: 0.42243367433547974
  y: 0.2916000783443451
  z: 0.013102240860462189
}
landmark {
  x: 0.4274732768535614
  y: 0.2724294364452362
  z: 0.013194995932281017
}
landmark {
  x: 0.4281640350818634
  y: 0.25694799423217773
  z: 0.01310694683343172
}
landmark {
  x: 0.43459638953208923
  y: 0.2697184979915619
  z: 0.012765987776219845
}
landmark {
  x: 0.4476584792137146
  y: 0.24204450845718384
  z: 0.011080027557909489
}
landmark {
  x: 0.45641160011291504
  y: 0.22330084443092346
  z: 0.009066078811883926
}
landmark {
  x: 0.46334460377693176
  y: 0.208185076713562
  z: 0.007697241846472025
}
landmark {
  x: 0.44003790616989136
  y: 0.2723827660083771
  z: 0.004574007820338011
}
landmark {
  x: 0.4553782343864441
  y: 0.24216842651367188
  z: 0.0029572718776762486
}
landmark {
  x: 0.46695613861083984
  y: 0.221754


# 데이터 축적하기 
4. Setup Folders for Collection


In [11]:
languages = ['가시','꽃', '나무']

In [13]:
#폴더 만들기 
#make a folder
import os
os.mkdir('MP_Data')

In [12]:
# 수화 단어 하나에 대한 폴더 만들기 
# make folder for each action in MP_Data 
import os
for i in languages:
    a = 'MP_Data//'+i
    os.mkdir(a)

In [13]:
# 하나의 동작 당 sequence를 담는 하위 폴더 생성 
#make folders contain each sequence data for each action (30 sequences)
for i in languages:
    a = 'MP_Data//'+i
    for k in range(0,30):
        b = a+'//'+str(k)
        os.mkdir(b)

    

In [14]:
#경로설정 
# Path for exported data, numpy arrays
DATA_PATH = os.path.join('MP_Data') 

# 추적할 action들 
# Actions that we try to detect
actions = np.array(languages)

#하나의 동작 당 실행 할 횟수
# Thirty videos worth of data
no_sequences = 30

#하나의 동영상 당 담을 frame 수 (30 frame)
# Videos are going to be 30 frames in length
sequence_length = 30

# Folder start
start_folder = 0

# 5. 데이터 쌓기 
5. Collect Keypoint Values for Training and Testing

In [15]:
import cv2
import sys
import time
from PIL import Image
from PIL import ImageDraw
from PIL import ImageFont
import numpy as np
import platform

# 한글 출력하는 함수 만들기 
def cv2_draw_label(image, text, point):
    x, y = point
    x, y = int(x), int(y)
    pil_image = Image.fromarray(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
    font = 'malgun.ttf' 
    draw = ImageDraw.Draw(pil_image)
    try:
        imageFont = ImageFont.truetype(font, 15) #글자크기 
    except:
        imageFont = ImageFont.load_default()
    draw.text((x, y), text, font=imageFont, fill=(0, 0, 255))  # 글자 색깔 
    image = cv2.cvtColor(np.array(pil_image), cv2.COLOR_RGB2BGR)

    return image

#비디오 녹화 시작 
cap = cv2.VideoCapture(0)
# Set mediapipe model 


with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:

    for action in actions:
        for sequence in range(start_folder, start_folder+no_sequences):
            for frame_num in range(sequence_length):


                ret, frame = cap.read()


                image, results = mediapipe_detection(frame, holistic)


                draw_styled_landmarks(image, results)
                
                if frame_num == 0: 
               # putText 는 text를 cv창에 바로 입력 해 주지만, 영어만 가능하다. 
                    cv2.putText(image, 'STARTING COLLECTION', (120,200), 
                               cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255, 0), 4, cv2.LINE_AA)
                    
                    #어디다가 저장하는 몇번째 sequence인지 
                    # cv창을 이미지로 변환시켜 한글을 출력하는 함수 적용하기 
                    image = cv2_draw_label(image, 'Collecting frames for {} Video Number {}'.format(action, sequence), (15,12))

                    cv2.imshow('OpenCV Feed', image)
                    cv2.waitKey(500)
                else: 
                    image = cv2_draw_label(image, 'Collecting frames for {} Video Number {}'.format(action, sequence), (15,12))

                    cv2.imshow('OpenCV Feed', image)
                
                # NEW Export keypoints
                # 필요한 data 추출한다. 
                keypoints = extract_keypoints(results)
                npy_path = os.path.join(DATA_PATH, action, str(sequence), str(frame_num))
                np.save(npy_path, keypoints)

                # Break gracefully
                # loop가 다 돌면 q를 누르고 종료한다. 
                if cv2.waitKey(10) & 0xFF == ord('q'):
                    break
                    
    cap.release()
    cv2.destroyAllWindows()

In [34]:
# 동영상 창을 끌 때 실행, 아니면 오류남 
# when you want to close the camara 

cap.release()
cv2.destroyAllWindows()

# 6. 데이터 라벨링하기 
6. Preprocess Data and Create Labels and Features

In [18]:
# 다른 액션에 대한 레이블 만들기 
# 필요한 데이터 임포트
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

In [19]:
# 정답데이터 라벨링 하기 
label_map = {label:num for num, label in enumerate(actions)} # enumerate-> 0 부터 순서대로 나타내 주는 것 

In [20]:
label_map

{'동물': 0, '동물원': 1, '사자': 2}

In [21]:
# 파일 불러와서 해당 영상에 대한 정답 데이터 labels에 담기 
# labeling actions to labels 

sequences, labels = [], []
for action in actions:
    for sequence in np.array(os.listdir(os.path.join(DATA_PATH, action))).astype(int):
        window = []
        for frame_num in range(sequence_length):
            # 데이터 파이썬 파일로 불러오기 
            res = np.load(os.path.join(DATA_PATH, action, str(sequence), "{}.npy".format(frame_num)))
            # 리스트에 append 하기 
            window.append(res)
        sequences.append(window)
        labels.append(label_map[action])

In [22]:
np.array(sequences).shape
# 총 90개의 비디오, 1편 당 30시퀀스 /  # (1662,)키포인트 -> 468*3 + 33*4 + 21*3 + 21*3 각(xyz 즉 세개씩 곱한다.)


(90, 30, 1662)

In [23]:
np.array(labels).shape  #(90,) 라벨링한 정답 데이터 

(90,)

In [24]:
X = np.array(sequences)

In [25]:
X.shape
#90 channels, 30 frames, 1662 keypoints (68*3 + 33*4 + 21*3 + 21*3)

(90, 30, 1662)

In [26]:
# 원핫인코딩 하기 
# one hot encoding 
y = to_categorical(labels).astype(int)

In [27]:
# 데이터 나누기
# split data 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05) # 5%가 테스트 되겠다

In [28]:
y_test.shape

(5, 3)

# LSTM Neural Network 만들기 

Build and Train LSTM Neural Network

In [30]:
# 모델 임포트 

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense  #LSTM layer 를  쓰겠다.  
from tensorflow.keras.callbacks import TensorBoard

In [31]:
# set call back ( prevent over fitting )
log_dir = os.path.join('Logs')
tb_callback = TensorBoard(log_dir=log_dir)

In [ ]:
# 이 타입 쓰는 이유 CNN 과 비교 시 장점 
# 미디아 type + LSTM 레이어
# 1. Less DaTa Required 
# 2. Faster To train
# 3. Faster Detections 

In [32]:
model = Sequential()  # 시작 
# 30, 1662) 각각의 비디오 30 시퀀스 1662 키포인트
model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(30,1662))) #64 유닛 -> LSTM은 다음 레이어에 sequences가 필요해서 다음 레이어가 있으면 True를 해준다
model.add(LSTM(128, return_sequences=True, activation='relu'))
model.add(LSTM(64, return_sequences=False, activation='relu'))# 다음은 dense슬라이드라 sequences 필요없어
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(actions.shape[0], activation='softmax')) # ctions.shape[0] -> 3네트워크 유닛 

In [33]:
# 아담 그레이드 , 비아너리 -> 바이너리 크로스 / 이건 분류하는 거-> multi class categoty  예측하는 확률을 보여준다. 
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [ ]:
model.fit(X_train, y_train, epochs=2000, callbacks=[tb_callback])

Epoch 1/2000


In [38]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 30, 64)            442112    
_________________________________________________________________
lstm_1 (LSTM)                (None, 30, 128)           98816     
_________________________________________________________________
lstm_2 (LSTM)                (None, 64)                49408     
_________________________________________________________________
dense (Dense)                (None, 64)                4160      
_________________________________________________________________
dense_1 (Dense)              (None, 32)                2080      
_________________________________________________________________
dense_2 (Dense)              (None, 3)                 99        
Total params: 596,675
Trainable params: 596,675
Non-trainable params: 0
__________________________________________________

 -  결과보기 
 - visualize the result of model



 cmd 창을 킨다 <br>
open cmd  <br>
<br>
C:\Users\User>cd Logs <br>
<br>
C:\Users\User\Logs>cd Train<br>
<br>
C:\Users\User\Logs\train>dir<br>

- 파일을 텐서플로우로 실행시킨다. (run the file )
C:\Users\User\Logs\train>tensorboard --logdir=. 

- 링크를 복사하여 크롬창에 띄운다 ( copy the like and open it at Chrome)

# 예측하기 
8. Make Predictions

In [39]:
res = model.predict(X_test)

In [53]:
actions[np.argmax(res[4])]

'hello'

In [54]:
actions[np.argmax(y_test[4])]

'iloveyou'

# 모델 저장 
9. Save Weights

In [51]:
model.save('action.h5')

In [59]:
model.load_weights('action.h5')

# 모델 평가 
10. Evaluation using Confusion Matrix and Accuracy

In [55]:
from sklearn.metrics import multilabel_confusion_matrix, accuracy_score

In [56]:
yhat = model.predict(X_train)

In [57]:
ytrue = np.argmax(y_train, axis=1).tolist()
yhat = np.argmax(yhat, axis=1).tolist()

In [58]:
multilabel_confusion_matrix(ytrue, yhat)
# 이원교차표 출력 

array([[[ 2, 54],
        [ 0, 29]],

       [[57,  0],
        [28,  0]],

       [[57,  0],
        [26,  2]]], dtype=int64)

In [59]:
accuracy_score(ytrue, yhat)

0.36470588235294116

# 실시간 예측하기 
11. Test in Real Time

In [64]:
from scipy import stats

In [65]:
colors = [(245,117,16), (117,245,16), (16,117,245)]
def prob_viz(res, actions, input_frame, colors):
    output_frame = input_frame.copy()
    for num, prob in enumerate(res):
        cv2.rectangle(output_frame, (0,60+num*40), (int(prob*100), 90+num*40), colors[num], -1)
        cv2.putText(output_frame, actions[num], (0, 85+num*40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
        
    return output_frame

In [66]:
plt.figure(figsize=(18,18))
plt.imshow(prob_viz(res, actions, image, colors))

TypeError: only size-1 arrays can be converted to Python scalars

<Figure size 1296x1296 with 0 Axes>

In [78]:
# 1. New detection variables #30개 차면 detect 할 수 있게
sequence = []
sentence = []
predictions = []
threshold = 0.5

cap = cv2.VideoCapture(0)
# Set mediapipe model 
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():

        # Read feed
        ret, frame = cap.read()

        # Make detections
        image, results = mediapipe_detection(frame, holistic)
        print(results)
        
        # Draw landmarks
        draw_styled_landmarks(image, results)
        
        # 2. Prediction logic
        keypoints = extract_keypoints(results)
        sequence.append(keypoints)
        sequence = sequence[-30:] #마지막 30프레임
        
        if len(sequence) == 30: #30개차면 predict한다 
            res = model.predict(np.expand_dims(sequence, axis=0))[0] # 2차원을 3차원으로 늘려준다. [0]
            print(actions[np.argmax(res)])
            predictions.append(np.argmax(res))
            
            
        #3. Viz logic
            if np.unique(predictions[-10:])[0]==np.argmax(res): 
                if res[np.argmax(res)] > threshold: 
                    
                    if len(sentence) > 0: 
                        if actions[np.argmax(res)] != sentence[-1]:
                            sentence.append(actions[np.argmax(res)])
                    else:
                        sentence.append(actions[np.argmax(res)])

            if len(sentence) > 5: 
                sentence = sentence[-5:]

            # Viz probabilities
            image = prob_viz(res, actions, image, colors)
            
        cv2.rectangle(image, (0,0), (640, 40), (245, 117, 16), -1)
        cv2.putText(image, ' '.join(sentence), (3,30), 
                       cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
        
        # Show to screen
        cv2.imshow('OpenCV Feed', image)

        # Break gracefully
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.soluti

<class 'mediapipe.python.solution_base.SolutionOutputs'>
thanks
<class 'mediapipe.python.solution_base.SolutionOutputs'>
thanks
<class 'mediapipe.python.solution_base.SolutionOutputs'>
hello
<class 'mediapipe.python.solution_base.SolutionOutputs'>
thanks
<class 'mediapipe.python.solution_base.SolutionOutputs'>
hello
<class 'mediapipe.python.solution_base.SolutionOutputs'>
thanks
<class 'mediapipe.python.solution_base.SolutionOutputs'>
thanks
<class 'mediapipe.python.solution_base.SolutionOutputs'>
iloveyou
<class 'mediapipe.python.solution_base.SolutionOutputs'>
iloveyou
<class 'mediapipe.python.solution_base.SolutionOutputs'>
iloveyou
<class 'mediapipe.python.solution_base.SolutionOutputs'>
hello
<class 'mediapipe.python.solution_base.SolutionOutputs'>
iloveyou
<class 'mediapipe.python.solution_base.SolutionOutputs'>
iloveyou
<class 'mediapipe.python.solution_base.SolutionOutputs'>
thanks
<class 'mediapipe.python.solution_base.SolutionOutputs'>
thanks
<class 'mediapipe.python.solution

thanks
<class 'mediapipe.python.solution_base.SolutionOutputs'>
iloveyou
<class 'mediapipe.python.solution_base.SolutionOutputs'>
iloveyou
<class 'mediapipe.python.solution_base.SolutionOutputs'>
thanks
<class 'mediapipe.python.solution_base.SolutionOutputs'>
thanks
<class 'mediapipe.python.solution_base.SolutionOutputs'>
iloveyou
<class 'mediapipe.python.solution_base.SolutionOutputs'>
iloveyou
<class 'mediapipe.python.solution_base.SolutionOutputs'>
iloveyou
<class 'mediapipe.python.solution_base.SolutionOutputs'>
iloveyou
<class 'mediapipe.python.solution_base.SolutionOutputs'>
iloveyou
<class 'mediapipe.python.solution_base.SolutionOutputs'>
iloveyou
<class 'mediapipe.python.solution_base.SolutionOutputs'>
iloveyou
<class 'mediapipe.python.solution_base.SolutionOutputs'>
iloveyou
<class 'mediapipe.python.solution_base.SolutionOutputs'>
iloveyou
<class 'mediapipe.python.solution_base.SolutionOutputs'>
iloveyou
<class 'mediapipe.python.solution_base.SolutionOutputs'>
iloveyou
<class 

<class 'mediapipe.python.solution_base.SolutionOutputs'>
thanks
<class 'mediapipe.python.solution_base.SolutionOutputs'>
thanks
<class 'mediapipe.python.solution_base.SolutionOutputs'>
thanks
<class 'mediapipe.python.solution_base.SolutionOutputs'>
thanks
<class 'mediapipe.python.solution_base.SolutionOutputs'>
thanks
<class 'mediapipe.python.solution_base.SolutionOutputs'>
thanks
<class 'mediapipe.python.solution_base.SolutionOutputs'>
thanks
<class 'mediapipe.python.solution_base.SolutionOutputs'>
thanks
<class 'mediapipe.python.solution_base.SolutionOutputs'>
thanks
<class 'mediapipe.python.solution_base.SolutionOutputs'>
thanks
<class 'mediapipe.python.solution_base.SolutionOutputs'>
thanks
<class 'mediapipe.python.solution_base.SolutionOutputs'>
thanks
<class 'mediapipe.python.solution_base.SolutionOutputs'>
thanks
<class 'mediapipe.python.solution_base.SolutionOutputs'>
thanks
<class 'mediapipe.python.solution_base.SolutionOutputs'>
thanks
<class 'mediapipe.python.solution_base.S

<class 'mediapipe.python.solution_base.SolutionOutputs'>
thanks
<class 'mediapipe.python.solution_base.SolutionOutputs'>
thanks
<class 'mediapipe.python.solution_base.SolutionOutputs'>
thanks
<class 'mediapipe.python.solution_base.SolutionOutputs'>
thanks
<class 'mediapipe.python.solution_base.SolutionOutputs'>
thanks
<class 'mediapipe.python.solution_base.SolutionOutputs'>
thanks
<class 'mediapipe.python.solution_base.SolutionOutputs'>
thanks
<class 'mediapipe.python.solution_base.SolutionOutputs'>
thanks
<class 'mediapipe.python.solution_base.SolutionOutputs'>
thanks
<class 'mediapipe.python.solution_base.SolutionOutputs'>
thanks
<class 'mediapipe.python.solution_base.SolutionOutputs'>
thanks
<class 'mediapipe.python.solution_base.SolutionOutputs'>
thanks
<class 'mediapipe.python.solution_base.SolutionOutputs'>
thanks
<class 'mediapipe.python.solution_base.SolutionOutputs'>
thanks
<class 'mediapipe.python.solution_base.SolutionOutputs'>
thanks
<class 'mediapipe.python.solution_base.S

<class 'mediapipe.python.solution_base.SolutionOutputs'>
thanks
<class 'mediapipe.python.solution_base.SolutionOutputs'>
thanks
<class 'mediapipe.python.solution_base.SolutionOutputs'>
thanks
<class 'mediapipe.python.solution_base.SolutionOutputs'>
thanks
<class 'mediapipe.python.solution_base.SolutionOutputs'>
thanks
<class 'mediapipe.python.solution_base.SolutionOutputs'>
thanks
<class 'mediapipe.python.solution_base.SolutionOutputs'>
thanks
<class 'mediapipe.python.solution_base.SolutionOutputs'>
thanks
<class 'mediapipe.python.solution_base.SolutionOutputs'>
thanks
<class 'mediapipe.python.solution_base.SolutionOutputs'>
thanks
<class 'mediapipe.python.solution_base.SolutionOutputs'>
thanks
<class 'mediapipe.python.solution_base.SolutionOutputs'>
thanks
<class 'mediapipe.python.solution_base.SolutionOutputs'>
thanks
<class 'mediapipe.python.solution_base.SolutionOutputs'>
thanks
<class 'mediapipe.python.solution_base.SolutionOutputs'>
thanks
<class 'mediapipe.python.solution_base.S

<class 'mediapipe.python.solution_base.SolutionOutputs'>
iloveyou
<class 'mediapipe.python.solution_base.SolutionOutputs'>
hello
<class 'mediapipe.python.solution_base.SolutionOutputs'>
iloveyou
<class 'mediapipe.python.solution_base.SolutionOutputs'>
iloveyou
<class 'mediapipe.python.solution_base.SolutionOutputs'>
iloveyou
<class 'mediapipe.python.solution_base.SolutionOutputs'>
iloveyou
<class 'mediapipe.python.solution_base.SolutionOutputs'>
iloveyou
<class 'mediapipe.python.solution_base.SolutionOutputs'>
iloveyou
<class 'mediapipe.python.solution_base.SolutionOutputs'>
iloveyou
<class 'mediapipe.python.solution_base.SolutionOutputs'>
iloveyou
<class 'mediapipe.python.solution_base.SolutionOutputs'>
hello
<class 'mediapipe.python.solution_base.SolutionOutputs'>
thanks
<class 'mediapipe.python.solution_base.SolutionOutputs'>
thanks
<class 'mediapipe.python.solution_base.SolutionOutputs'>
thanks
<class 'mediapipe.python.solution_base.SolutionOutputs'>
thanks
<class 'mediapipe.python

In [74]:
cap.release()
cv2.destroyAllWindows()